In [14]:
import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np


'''
Hinweis
Bei AttributeError: module 'tensorflow_core.python.keras.api._v2.keras.activations' has no attribute 'swish'
der Fehler tritt nicht auf/ lässt sich beheben wenn der python 3.8 Kernel ausgewählt ist.
'''
from transformers import BertForSequenceClassification, AdamW, BertConfig

df = pd.read_csv('one_label_one_hot_nkm_open.csv',index_col=0)
num_labels = len(df.columns)-1

BertTokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-german-dbmdz-cased')

# vortrainiertes model!
#BERTmodel = torch.hub.load('huggingface/pytorch-transformers', 'model', 'bert-base-german-dbmdz-cased')

BERTmodel =  BertForSequenceClassification.from_pretrained(
    "bert-base-german-dbmdz-cased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = num_labels, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# args.learning_rate - default is 5e-5, our notebook had 2e-5'
# # args.adam_epsilon  - default is 1e-8.

optimizer = AdamW(BERTmodel.parameters(),lr = 2e-5, eps = 1e-8 )


# load
device = torch.device('cpu')
checkpoint = torch.load('one_hot_nkm_bert.pth', map_location='cpu')
BERTmodel.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
BERTmodel.eval()

Using cache found in /home/azureuser/.cache/torch/hub/huggingface_pytorch-transformers_master
Some weights of the model checkpoint at bert-base-german-dbmdz-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassificatio

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31102, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [30]:
import tensorflow as tf
import numpy as np
text = 'Ich habe Nichts anzumerken.'
tokens = BertTokenizer.tokenize(text)
print(tokens)

# added start and end token and convert to ids
tokens = ["[CLS]"] + tokens + ["[SEP]"]
input_ids = BertTokenizer.convert_tokens_to_ids(tokens)
print(input_ids)

# put input on gpu and make prediction
bert_output = BERTmodel(torch.tensor([input_ids]).cpu())
logits = bert_output.logits.detach().cpu()
print(bert_output)
print('\n\n')

print(np.argmax(tf.nn.softmax(logits)), logits, "argmax ", logits.argmax(), np.argmax(logits[0] ))


['Ich', 'habe', 'Nichts', 'anzu', '##merk', '##en', '.']
[102, 395, 704, 8009, 4276, 10795, 106, 566, 103]
SequenceClassifierOutput(loss=None, logits=tensor([[ 0.1182,  1.3392,  2.1417, -1.5125, -1.7348, -1.9602, -1.5444, -1.1416,
         -0.6820,  3.3628, -1.6096,  1.2417, -0.7303, -0.9200, -1.7151, -0.5717,
          0.3418]], grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)



9 tensor([[ 0.1182,  1.3392,  2.1417, -1.5125, -1.7348, -1.9602, -1.5444, -1.1416,
         -0.6820,  3.3628, -1.6096,  1.2417, -0.7303, -0.9200, -1.7151, -0.5717,
          0.3418]]) tf.Tensor(
[[0.5295227  0.79235905 0.8948889  0.18057123 0.14997667 0.12344512
  0.17589244 0.24201989 0.33581644 0.9665231  0.16664702 0.775865
  0.32513595 0.28495026 0.15249822 0.36085606 0.584619  ]], shape=(1, 17), dtype=float32) argmax  tensor(9) tensor(9)


In [ ]:

def write_predictions_to_origin(FILE_PATH, preds, labels):



    df = pd.read_csv(FILE_PATH,index_col=0)
    # list of tuples
    lot = []

    pred_flat = preds
    labels_flat = labels

    for pred, label in zip(pred_flat, labels_flat):
        lot.append((pred, label))
    df['prediction-label'] = lot
    df.to_csv('output.csv', index=False)




write_predictions_to_origin('one_label_one_hot_nkm_open.csv', predictions, labels)